#STEP 1: Setting Up the Project

In [18]:
"""
HomeGuard Security System Simulator
Author: Marco Martins
Description: A smart home monitoring system that processes sensor readings
             and triggers alerts for security, safety, and comfort issues.
"""

import random
from datetime import datetime

# System configuration
HOME_MODES = ["HOME", "AWAY", "SLEEP"]
ALERT_SEVERITIES = ["LOW", "MEDIUM", "HIGH", "CRITICAL"]

# Current system state
current_mode = "AWAY"


#STEP : Creating Data Structures

In [20]:
def create_sensor(sensor_id, location, sensor_type, threshold=None):
    """
    Creates a sensor data structure.
    
    Parameters:
    - sensor_id: Unique identifier for the sensor
    - location: Where the sensor is located (e.g., "Living Room")
    - sensor_type: Type of sensor ("motion", "temperature", "door", "smoke")
    - threshold: Optional threshold value for the sensor
    
    Returns:
    - A dictionary representing the sensor
    """
    sensor = {
        "id": sensor_id,
        "location": location,
        "type": sensor_type,
        "current_value": None,
        "threshold": threshold
    }
    return sensor

def create_alert(severity, message, sensor_id, timestamp):
    """
    Creates an alert data structure.
    
    Parameters:
    - severity: Alert severity level (LOW, MEDIUM, HIGH, CRITICAL)
    - message: Description of the alert
    - sensor_id: ID of the sensor that triggered the alert
    - timestamp: When the alert was triggered
    
    Returns:
    - A dictionary representing the alert
    """
    alert = {
        "severity": severity,
        "message": message,
        "sensor_id": sensor_id,
        "timestamp": timestamp
    }
    return alert

# Initialize sensors for the Peterson home
sensors = [
    create_sensor(1, "Living Room", "motion"),
    create_sensor(2, "Kitchen", "temperature", threshold=35),
    create_sensor(3, "Front Door", "door"),
    create_sensor(4, "Bedroom", "smoke")
]


In [21]:
#Test sensors are created
print(f"Initialized {len(sensors)} sensors")
for sensor in sensors:
    print(f"  - {sensor['id']}: {sensor['location']} ({sensor['type']})")


Initialized 4 sensors
  - 1: Living Room (motion)
  - 2: Kitchen (temperature)
  - 3: Front Door (door)
  - 4: Bedroom (smoke)


Step 3: Implementing If/Else Logic

In [22]:
def is_abnormal_reading(sensor, reading_value):
    """
    Checks if a sensor reading is abnormal based on sensor type and thresholds.
    
    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The current reading from the sensor
    
    Returns:
    - True if the reading is abnormal, False otherwise
    """
    sensor_type = sensor["type"]
    
    if sensor_type == "temperature":
        if reading_value < 35 or reading_value > 95:
            return True
    elif sensor_type == "motion":
        if reading_value == True:  # motion detected
            return True
    elif sensor_type == "door":
        if reading_value == "OPEN":
            return True
    elif sensor_type == "smoke":
        if reading_value == "DETECTED":
            return True
    
    return False


def should_trigger_security_alert(sensor, reading_value, system_mode):
    """
    Determines if a security alert should be triggered.
    
    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The current reading from the sensor
    - system_mode: Current system mode (HOME, AWAY, SLEEP)
    
    Returns:
    - True if a security alert should be triggered, False otherwise
    """
    sensor_type = sensor["type"]
    
    # Security alerts only relevant in AWAY or SLEEP mode
    if system_mode in ["AWAY", "SLEEP"]:
        if sensor_type == "motion" and reading_value == True:
            return True
        if sensor_type == "door" and reading_value == "OPEN":
            return True
    
    # Smoke and extreme temperature are always considered safety alerts
    if sensor_type == "smoke" and reading_value == "DETECTED":
        return True
    if sensor_type == "temperature" and (reading_value < 35 or reading_value > 95):
        return True
    
    return False


In [23]:
#TEST
# Test temperature check
test_sensor = create_sensor("TEMP_TEST", "Test Room", "temperature", threshold=35)
print(f"34°F is abnormal: {is_abnormal_reading(test_sensor, 34)}")  # Should be True
print(f"68°F is abnormal: {is_abnormal_reading(test_sensor, 68)}")  # Should be False

# Test security alert
motion_sensor = create_sensor("MOTION_TEST", "Test Room", "motion")
print(f"Motion in AWAY mode triggers alert: {should_trigger_security_alert(motion_sensor, True, 'AWAY')}")  # Should be True
print(f"Motion in HOME mode triggers alert: {should_trigger_security_alert(motion_sensor, True, 'HOME')}")  # Should be False


34°F is abnormal: True
68°F is abnormal: False
Motion in AWAY mode triggers alert: True
Motion in HOME mode triggers alert: False


Step 4: Building Functions

In [24]:
def generate_reading(sensor):
    """
    Generates a realistic reading for a sensor.
    
    Parameters:
    - sensor: Sensor dictionary
    
    Returns:
    - A realistic reading value based on sensor type
    """    
    if sensor["type"] == "temperature":
        return random.randint(30, 100)  # Fahrenheit
    elif sensor["type"] == "motion":
        return random.choice([True, False])
    elif sensor["type"] == "door":
        return random.choice(["OPEN", "CLOSED"])
    elif sensor["type"] == "smoke":
        return random.choice(["CLEAR", "DETECTED"])
    else:
        return None


def process_reading(sensor, reading_value, system_mode):
    """
    Processes a sensor reading and determines if alerts are needed.
    
    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The reading from the sensor
    - system_mode: Current system mode
    
    Returns:
    - A list of alerts (empty if no alerts needed)
    """
    alerts = []
    timestamp = datetime.now().strftime("%H:%M:%S")
    
    # Update the sensor's current value
    sensor["current_value"] = reading_value
    
    # Security alerts (motion/door in AWAY or SLEEP mode)
    if should_trigger_security_alert(sensor, reading_value, system_mode):
        severity = "HIGH" if sensor["type"] in ["motion", "door"] else "CRITICAL"
        message = f"{sensor['type'].capitalize()} alert at {sensor['location']}"
        alert = create_alert(severity, message, sensor["id"], timestamp)
        alerts.append(alert)
    
    # Safety alerts (temperature extremes, smoke)
    if sensor["type"] == "temperature" and (reading_value < 35 or reading_value > 95):
        alert = create_alert("CRITICAL", f"Temperature abnormal at {sensor['location']} ({reading_value}°F)", sensor["id"], timestamp)
        alerts.append(alert)
    if sensor["type"] == "smoke" and reading_value == "DETECTED":
        alert = create_alert("CRITICAL", f"Smoke detected at {sensor['location']}", sensor["id"], timestamp)
        alerts.append(alert)
    
    # Comfort notifications (temperature out of range in HOME mode)
    if sensor["type"] == "temperature" and system_mode == "HOME" and not (65 <= reading_value <= 75):
        alert = create_alert("LOW", f"Temperature out of comfort range at {sensor['location']} ({reading_value}°F)", sensor["id"], timestamp)
        alerts.append(alert)
    
    return alerts


def trigger_alert(alert):
    """
    Displays an alert to the user.
    
    Parameters:
    - alert: Alert dictionary
    """
    severity_symbol = {
        "LOW": "ℹ️",
        "MEDIUM": "⚠️",
        "HIGH": "🚨",
        "CRITICAL": "🔥"
    }
    
    symbol = severity_symbol.get(alert["severity"], "⚠️")
    print(f"[ALERT!] {symbol} {alert['severity']}: {alert['message']}")


def log_event(message, timestamp=None):
    """
    Logs an event to the console.
    
    Parameters:
    - message: The message to log
    - timestamp: Optional timestamp (uses current time if not provided)
    """
    if timestamp is None:
        timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[LOG] [{timestamp}] {message}")


In [25]:
#TEST
# Test reading generation
test_sensor = sensors[0]  # Motion sensor
reading = generate_reading(test_sensor)
print(f"Generated reading for {test_sensor['location']}: {reading}")

# Test processing
alerts = process_reading(test_sensor, True, "AWAY")
if alerts:
    trigger_alert(alerts[0])


Generated reading for Living Room: False
[ALERT!] 🚨 HIGH: Motion alert at Living Room


Step 5: Creating Classes

In [27]:
import random

class Sensor:
    """
    Represents a sensor in the HomeGuard system.
    """
    
    def __init__(self, sensor_id, location, sensor_type, threshold=None):
        """
        Initializes a new sensor.
        
        Parameters:
        - sensor_id: Unique identifier for the sensor
        - location: Where the sensor is located
        - sensor_type: Type of sensor ("motion", "temperature", "door", "smoke")
        - threshold: Optional threshold value for the sensor
        """
        self.id = sensor_id
        self.location = location
        self.type = sensor_type
        self.threshold = threshold
        self.current_value = None
    
    def read(self):
        """
        Generates and stores a new reading for this sensor.
        Returns the reading value.
        """
        if self.type == "temperature":
            self.current_value = random.uniform(20, 45)  # realistic temp range
        elif self.type == "motion":
            self.current_value = random.choice([True, False])
        elif self.type == "door":
            self.current_value = random.choice(["open", "closed"])
        elif self.type == "smoke":
            self.current_value = random.uniform(0, 10)  # smoke level
        return self.current_value
    
    def isAbnormal(self):
        """
        Checks if the current reading is abnormal.
        Returns True if abnormal, False otherwise.
        """
        if self.current_value is None:
            return False
        
        if self.type == "temperature" and self.threshold is not None:
            return self.current_value > self.threshold
        elif self.type == "motion":
            # example logic: motion detected at night might be abnormal
            return self.current_value is True
        elif self.type == "door":
            # example: door open might be abnormal during night
            return self.current_value == "open"
        elif self.type == "smoke":
            return self.current_value > 5  # arbitrary smoke threshold
        return False
    
    def reset(self):
        """Resets the sensor's current reading to None."""
        self.current_value = None
    
    def __str__(self):
        """Returns a string representation of the sensor."""
        status = "No reading" if self.current_value is None else str(self.current_value)
        return f"{self.id} ({self.location}): {status}"


# Example: create sensor objects
sensor_objects = [
    Sensor("MOTION_001", "Living Room", "motion"),
    Sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
    Sensor("DOOR_001", "Front Door", "door"),
    Sensor("SMOKE_001", "Bedroom", "smoke")
]




In [28]:
#TEST
# Test: reading and checking abnormal values
for sensor in sensor_objects:
    sensor.read()
    print(sensor, "Abnormal?", sensor.isAbnormal())

MOTION_001 (Living Room): False Abnormal? False
TEMP_001 (Kitchen): 39.56186364492035 Abnormal? True
DOOR_001 (Front Door): closed Abnormal? False
SMOKE_001 (Bedroom): 4.285091469502684 Abnormal? False


#Step 6: Integrate Everything

In [1]:
from datetime import datetime  # For timestamps in the simulation
import time  # To add delays between simulation steps
import random  # To generate random sensor readings

# --- Sensor class ---
class Sensor:
    """
    Represents a sensor in the HomeGuard system.
    Each sensor has an ID, location, type, optional threshold, and current reading.
    """
    
    def __init__(self, sensor_id, location, sensor_type, threshold=None):
        # Unique identifier for the sensor
        self.id = sensor_id
        # Location of the sensor (e.g., "Living Room")
        self.location = location
        # Type of sensor: motion, temperature, door, or smoke
        self.type = sensor_type
        # Threshold value for abnormal readings (if applicable)
        self.threshold = threshold
        # Current reading of the sensor (initially None)
        self.current_value = None
    
    def read(self):
        """
        Generate a new sensor reading based on its type.
        Returns the current reading.
        """
        if self.type == "temperature":
            # Random temperature between 60 and 80 °F
            self.current_value = random.uniform(60, 80)
        elif self.type == "motion":
            # Randomly True (motion detected) or False (no motion)
            self.current_value = random.choice([True, False])
        elif self.type == "door":
            # Randomly "open" or "closed"
            self.current_value = random.choice(["open", "closed"])
        elif self.type == "smoke":
            # Random smoke level between 0 and 10
            self.current_value = random.uniform(0, 10)
        return self.current_value
    
    def isAbnormal(self):
        """
        Check if the current reading is abnormal.
        Returns True if abnormal, False otherwise.
        """
        if self.current_value is None:
            return False
        if self.type == "temperature" and self.threshold:
            return self.current_value > self.threshold
        elif self.type == "motion":
            return self.current_value is True
        elif self.type == "door":
            return self.current_value == "open"
        elif self.type == "smoke":
            return self.current_value > 5
        return False
    
    def reset(self):
        """Reset the sensor's reading to None."""
        self.current_value = None
    
    def __str__(self):
        """Return a readable string showing sensor ID, location, and current value."""
        status = "No reading" if self.current_value is None else str(self.current_value)
        return f"{self.id} ({self.location}): {status}"


# --- Functions to handle alerts and logging ---
def process_reading(sensor_dict, value, system_mode):
    """
    Process a sensor reading and generate alerts if needed.
    
    Parameters:
    - sensor_dict: dictionary with sensor info
    - value: current sensor reading
    - system_mode: current mode (HOME, AWAY, SLEEP)
    
    Returns:
    - List of alerts generated based on sensor type and reading
    """
    alerts = []
    
    # Temperature alert if value exceeds 75°F
    if sensor_dict["type"] == "temperature" and value > 75:
        alerts.append({
            "sensor": sensor_dict["id"],
            "severity": "MEDIUM",
            "message": "High temperature!"
        })
    
    # Smoke alert if value exceeds 5
    if sensor_dict["type"] == "smoke" and value > 5:
        alerts.append({
            "sensor": sensor_dict["id"],
            "severity": "HIGH",
            "message": "Smoke detected!"
        })
    
    # Motion alert only triggers when system is in AWAY mode
    if sensor_dict["type"] == "motion" and value is True and system_mode == "AWAY":
        alerts.append({
            "sensor": sensor_dict["id"],
            "severity": "CRITICAL",
            "message": "Motion detected while away!"
        })
    
    return alerts

def trigger_alert(alert):
    """Display an alert in the console."""
    print(f"[ALERT - {alert['severity']}] {alert['sensor']}: {alert['message']}")

def log_event(message):
    """Log a message in the console."""
    print(f"[LOG] {message}")


# --- Main simulation function ---
def run_simulation(duration_minutes=5, system_mode="AWAY"):
    """
    Run the HomeGuard system simulation.
    
    Parameters:
    - duration_minutes: number of simulation iterations (each simulates a minute)
    - system_mode: current mode of the system (HOME, AWAY, SLEEP)
    """
    # Header for the simulation
    print("=" * 50)
    print("=== HomeGuard Security System ===")
    print("=" * 50)
    print(f"Mode: {system_mode}\n")
    
    # Create all sensor objects
    sensors = [
        Sensor("MOTION_001", "Living Room", "motion"),
        Sensor("TEMP_001", "Kitchen", "temperature", threshold=75),
        Sensor("DOOR_001", "Front Door", "door"),
        Sensor("SMOKE_001", "Bedroom", "smoke")
    ]
    
    # Loop to simulate each minute
    for minute in range(duration_minutes):
        # Current timestamp
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"\nTime: {current_time}")
        
        # Read all sensors and display readings
        for sensor in sensors:
            reading = sensor.read()
            
            # Display sensor reading with status
            if sensor.type == "temperature":
                status = "Normal" if reading <= sensor.threshold else "Abnormal"
                print(f"[READING] {sensor.location} Temperature: {reading:.1f}°F ({status})")
            elif sensor.type == "motion":
                status = "DETECTED" if reading else "No activity"
                print(f"[READING] {sensor.location} Motion: {status}")
            elif sensor.type == "door":
                print(f"[READING] {sensor.location} Door: {reading}")
            elif sensor.type == "smoke":
                status = "Abnormal" if reading > 5 else "Normal"
                print(f"[READING] {sensor.location} Smoke: {reading:.1f} ({status})")
            
            # Convert sensor to dictionary to process alerts
            sensor_dict = {
                "id": sensor.id,
                "location": sensor.location,
                "type": sensor.type,
                "threshold": sensor.threshold
            }
            alerts = process_reading(sensor_dict, reading, system_mode)
            
            # Trigger alerts and log important events
            for alert in alerts:
                trigger_alert(alert)
                if alert["severity"] in ["HIGH", "CRITICAL"]:
                    log_event("Sending notification to homeowner...")
        
        # Delay to make simulation readable
        time.sleep(1)  # Each iteration simulates 1 minute (adjust as needed) 
    
    # Simulation finished
    print("\n" + "=" * 50)
    print("Simulation complete!")
    print("=" * 50)


# --- Run the simulation ---
if __name__ == "__main__":
    # Run the simulator for 3 "minutes" in AWAY mode
    run_simulation(duration_minutes=3, system_mode="AWAY")


=== HomeGuard Security System ===
Mode: AWAY


Time: 15:02:54
[READING] Living Room Motion: DETECTED
[ALERT - CRITICAL] MOTION_001: Motion detected while away!
[LOG] Sending notification to homeowner...
[READING] Kitchen Temperature: 74.5°F (Normal)
[READING] Front Door Door: closed
[READING] Bedroom Smoke: 4.0 (Normal)

Time: 15:02:55
[READING] Living Room Motion: DETECTED
[ALERT - CRITICAL] MOTION_001: Motion detected while away!
[LOG] Sending notification to homeowner...
[READING] Kitchen Temperature: 66.9°F (Normal)
[READING] Front Door Door: closed
[READING] Bedroom Smoke: 9.2 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Time: 15:02:56
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 64.0°F (Normal)
[READING] Front Door Door: closed
[READING] Bedroom Smoke: 0.5 (Normal)

Simulation complete!


#Step 7: Testing different scenarios

Testing Different System Modes:

1) Security Test: Run with system_mode="AWAY" and verify security alerts trigger
2) Safety Test: Check that temperature and smoke alerts work
3) Comfort Test: Run with system_mode="HOME" and verify comfort notifications appea

Mode	What to check
AWAY	Motion or door opens → security alert should trigger
HOME	Comfort notifications (like high temp or minor smoke) should appear, but security alerts are relaxed
SLEEP	Night-time logic: doors open, motion detected → medium alerts

In [2]:
#Security Test (AWAY)
print("\n=== Security Test: AWAY mode ===")
run_simulation(duration_minutes=3, system_mode="AWAY")


=== Security Test: AWAY mode ===
=== HomeGuard Security System ===
Mode: AWAY


Time: 15:03:30
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 71.8°F (Normal)
[READING] Front Door Door: open
[READING] Bedroom Smoke: 1.3 (Normal)

Time: 15:03:31
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 80.0°F (Abnormal)
[ALERT - MEDIUM] TEMP_001: High temperature!
[READING] Front Door Door: open
[READING] Bedroom Smoke: 8.7 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Time: 15:03:32
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 63.7°F (Normal)
[READING] Front Door Door: open
[READING] Bedroom Smoke: 7.9 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Simulation complete!


In [3]:
#Safety Test (Temperature & Smoke)
print("\n=== Safety Test: HOME mode for safety ===")
run_simulation(duration_minutes=3, system_mode="HOME")


=== Safety Test: HOME mode for safety ===
=== HomeGuard Security System ===
Mode: HOME


Time: 15:03:39
[READING] Living Room Motion: DETECTED
[READING] Kitchen Temperature: 76.2°F (Abnormal)
[ALERT - MEDIUM] TEMP_001: High temperature!
[READING] Front Door Door: open
[READING] Bedroom Smoke: 6.2 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Time: 15:03:40
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 66.6°F (Normal)
[READING] Front Door Door: closed
[READING] Bedroom Smoke: 8.5 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Time: 15:03:41
[READING] Living Room Motion: DETECTED
[READING] Kitchen Temperature: 73.0°F (Normal)
[READING] Front Door Door: open
[READING] Bedroom Smoke: 8.3 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Simulation complete!


In [4]:
#Comfort Test (HOME)
print("\n=== Comfort Test: HOME mode ===")
run_simulation(duration_minutes=3, system_mode="HOME")


=== Comfort Test: HOME mode ===
=== HomeGuard Security System ===
Mode: HOME


Time: 15:03:51
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 63.8°F (Normal)
[READING] Front Door Door: closed
[READING] Bedroom Smoke: 4.6 (Normal)

Time: 15:03:52
[READING] Living Room Motion: DETECTED
[READING] Kitchen Temperature: 77.5°F (Abnormal)
[ALERT - MEDIUM] TEMP_001: High temperature!
[READING] Front Door Door: closed
[READING] Bedroom Smoke: 7.9 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Time: 15:03:53
[READING] Living Room Motion: DETECTED
[READING] Kitchen Temperature: 73.3°F (Normal)
[READING] Front Door Door: open
[READING] Bedroom Smoke: 9.9 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Simulation complete!


In [5]:
#Test for all 3 scenarios
# List of scenarios you want to test
scenarios = ["HOME", "AWAY", "SLEEP"]

# Run simulation for each scenario
for mode in scenarios:
    print(f"\n=== Simulation Test: {mode} Mode ===")
    run_simulation(duration_minutes=3, system_mode=mode)


=== Simulation Test: HOME Mode ===
=== HomeGuard Security System ===
Mode: HOME


Time: 15:04:04
[READING] Living Room Motion: DETECTED
[READING] Kitchen Temperature: 61.4°F (Normal)
[READING] Front Door Door: closed
[READING] Bedroom Smoke: 9.8 (Abnormal)
[ALERT - HIGH] SMOKE_001: Smoke detected!
[LOG] Sending notification to homeowner...

Time: 15:04:05
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 70.7°F (Normal)
[READING] Front Door Door: open
[READING] Bedroom Smoke: 0.4 (Normal)

Time: 15:04:06
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 65.2°F (Normal)
[READING] Front Door Door: open
[READING] Bedroom Smoke: 1.5 (Normal)

Simulation complete!

=== Simulation Test: AWAY Mode ===
=== HomeGuard Security System ===
Mode: AWAY


Time: 15:04:07
[READING] Living Room Motion: No activity
[READING] Kitchen Temperature: 72.5°F (Normal)
[READING] Front Door Door: closed
[READING] Bedroom Smoke: 6.8 (Abnormal)
[ALERT - HIGH] SMOKE_001